####  2세대 도감 만들기 
<hr>
1. 251 마리의 이미지를 받아서 <br>
2. 70 * 70 사이즈로 리사이즈 <br>
3. 컬러로 돌릴 예정   약 4900*3 = 150000 컬럼 예상<br>
4. 이미지 251개를 10배가량 복제할 예정<br>
5. 이미지 처리후, csv에 저장. <br>
<hr>
6. csv로부터 파일을 읽어서 도감 번호를 타겟으로 설정 하여 트레인/테스트<br>
7. 모델 만들기.<br>
8. 여유있으면 웹연결하여 예측모델 만들기.<br>




In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import seaborn as sns
import pandas as pd # 데이터 분석 및 전처리

from _utils.ClassfierModule import ClassfierModule as cm
from _utils import tools as ts

In [50]:
data_dir = './_data/'
img_dir  = './_image/'


In [51]:
initDF = pd.DataFrame(pd.read_csv(data_dir+'sd_poke', index_col= 0, encoding='utf-8'))

In [52]:
initDF.head()

,0,1,2,3,4,5,6,7,8,9,...,14691,14692,14693,14694,14695,14696,14697,14698,14699,14700
1520,152,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1520i,152,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
15230,152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15230i,152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15260,152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
# initDF.loc[initDF.index[1:], '0'] = initDF.loc[initDF.index[1:], '0']*255
# initDF.loc[initDF.index[1:], '0'] = initDF.loc[initDF.index[1:], '0'].astype(int)

In [54]:
initDF.loc[initDF.index[1:], '0'].head()

1520i     152
15230     152
15230i    152
15260     152
15260i    152
Name: 0, dtype: int64

In [55]:
initDF['0'] = initDF['0'].astype(int)

In [56]:
initDF['0']

1520       152
1520i      152
15230      152
15230i     152
15260      152
          ... 
251300i    251
251330     251
251330i    251
251360     251
251360i    251
Name: 0, Length: 2600, dtype: int32

In [57]:
X_train, X_test, y_train, y_test = ts.train_test_cut(initDF.loc[:,'1':], initDF['0'], stratify=initDF['0'])

X_train => 2D (1950, 14700) / X_test => 1D, (1950,)
y_train => 2D (650, 14700), / y_test => 1D, (650,)


In [58]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [59]:
test = cm(X_train, y_train, X_test, y_test)

In [60]:
dtModel, dtDF = test.decision_tree_classification(max_depth=20, min_samples_split=4, min_samples_leaf = 2)
dtDF

              precision    recall  f1-score   support

         152       0.86      0.86      0.86         7
         153       0.80      0.57      0.67         7
         154       0.40      0.29      0.33         7
         155       0.42      0.83      0.56         6
         156       1.00      1.00      1.00         7
         157       0.56      0.71      0.62         7
         158       0.75      1.00      0.86         6
         159       1.00      1.00      1.00         7
         160       0.75      0.86      0.80         7
         161       0.71      0.71      0.71         7
         162       0.56      0.83      0.67         6
         163       0.60      0.50      0.55         6
         164       0.42      0.83      0.56         6
         165       0.50      0.50      0.50         6
         166       1.00      0.67      0.80         6
         167       0.78      1.00      0.88         7
         168       0.00      0.00      0.00         7
         169       0.62    

c:\Users\matdo\anaconda3\envs\ML_CV\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\matdo\anaconda3\envs\ML_CV\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\matdo\anaconda3\envs\ML_CV\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,train_score,test_score,diff,train_loss,test_loss
dt,0.880513,0.683077,0.197436,0.119487,0.316923


In [61]:
rfModel, rfDF = test.rf_classifier(max_depth=20, min_samples_split=4, min_samples_leaf = 2)
rfDF

              precision    recall  f1-score   support

         152       1.00      1.00      1.00         7
         153       1.00      1.00      1.00         7
         154       1.00      1.00      1.00         7
         155       1.00      1.00      1.00         6
         156       1.00      1.00      1.00         7
         157       1.00      1.00      1.00         7
         158       1.00      1.00      1.00         6
         159       1.00      1.00      1.00         7
         160       1.00      1.00      1.00         7
         161       0.78      1.00      0.88         7
         162       1.00      1.00      1.00         6
         163       1.00      1.00      1.00         6
         164       0.86      1.00      0.92         6
         165       1.00      1.00      1.00         6
         166       1.00      0.83      0.91         6
         167       1.00      1.00      1.00         7
         168       1.00      1.00      1.00         7
         169       1.00    

,train_score,test_score,diff,train_loss,test_loss
rf,1.0,0.986154,0.013846,0.0,0.013846


In [62]:
import joblib
joblib.dump(rfModel, "./_model/rf_model.joblib")

['./_model/rf_model.joblib']